In [30]:
import numpy as np

In [31]:
tau_relax = 27
delta_tau = 5
P = 23
eta = np.random.uniform(0.01, 0.5)

# The direct path tau_p = 0 
tau_p = [0]

# Generate remaining delays tau_p using uniform distribution, excluding zero, allowing repeats
remaining_tau_p = np.random.randint(1, tau_relax - delta_tau + 1, P - 1)

# Combine and sort 
tau_p = np.concatenate((tau_p, remaining_tau_p))
tau_p = np.sort(tau_p)

# Calculate amplitude gains alpha_p of h_tau_p
alpha_p = np.exp(-eta * tau_p / 2)

# Generate random phases phi_p
phi_p = np.random.uniform(0, 2 * np.pi, P)

# Calculate complex gains h_tau_p
h_tau_p = alpha_p * np.exp(1j * phi_p)

for p in range(P):
    print(f"Path {p + 1}: tau_p = {tau_p[p]}, alpha_p = {alpha_p[p]:.4f}, phi_p = {phi_p[p]:.4f}, h_tau_p = {h_tau_p[p]:.4f}")


Path 1: tau_p = 0, alpha_p = 1.0000, phi_p = 5.5529, h_tau_p = 0.7450-0.6670j
Path 2: tau_p = 3, alpha_p = 0.6150, phi_p = 2.4691, h_tau_p = -0.4811+0.3831j
Path 3: tau_p = 5, alpha_p = 0.4448, phi_p = 5.8109, h_tau_p = 0.3961-0.2023j
Path 4: tau_p = 5, alpha_p = 0.4448, phi_p = 1.5810, h_tau_p = -0.0045+0.4448j
Path 5: tau_p = 7, alpha_p = 0.3217, phi_p = 0.9920, h_tau_p = 0.1760+0.2693j
Path 6: tau_p = 9, alpha_p = 0.2327, phi_p = 4.9440, h_tau_p = 0.0534-0.2264j
Path 7: tau_p = 11, alpha_p = 0.1683, phi_p = 6.1136, h_tau_p = 0.1658-0.0284j
Path 8: tau_p = 11, alpha_p = 0.1683, phi_p = 0.3898, h_tau_p = 0.1556+0.0639j
Path 9: tau_p = 12, alpha_p = 0.1431, phi_p = 0.6283, h_tau_p = 0.1158+0.0841j
Path 10: tau_p = 12, alpha_p = 0.1431, phi_p = 1.9974, h_tau_p = -0.0592+0.1303j
Path 11: tau_p = 13, alpha_p = 0.1217, phi_p = 3.3046, h_tau_p = -0.1201-0.0198j
Path 12: tau_p = 14, alpha_p = 0.1035, phi_p = 4.4251, h_tau_p = -0.0293-0.0992j
Path 13: tau_p = 15, alpha_p = 0.0880, phi_p = 6.0

In [32]:
N = 128  # Number of sub-carriers
Lc = 32  # Length of cyclic prefix

# Step 1: Generate the data symbols d_k
np.random.seed(0)  
d_k = np.random.randn(N) + 1j * np.random.randn(N)  # Generate random complex symbols

# Step 2: Calculate the time-domain samples s_n using IDFT
s_n = np.zeros(N, dtype=complex)

for n in range(N):
    s_n[n] = (1/N) * np.sum(d_k * np.exp(1j * 2 * np.pi * np.arange(N) * n / N))

# Step 3: Append the cyclic prefix (CP)
cp = s_n[-Lc:]  # Last Lc samples of s_n
s_cp = np.concatenate((cp, s_n))  # Append CP to the beginning of s_n

print("Data symbols d_k:")
print(len(d_k))
print("\nTime-domain samples s_n:")
print(len(s_n))
print("\nTime-domain samples with cyclic prefix s_cp:")
print(len(s_cp))


Data symbols d_k:
128

Time-domain samples s_n:
128

Time-domain samples with cyclic prefix s_cp:
160


In [33]:
Nu = N + Lc
M = Nu + N
P = 23
epsilon = 0.1  # Example carrier frequency offset (CFO)

# Generate time offset
tilde_tau = np.random.randint(0, N)

# Generate M-length received signal
r_n = np.zeros(M, dtype=complex)
w_n = np.random.randn(M) + 1j * np.random.randn(M)  # AWGN

for n in range(M):
    r_n[n] = np.sum([h_tau_p[p] * s_cp[(n - tilde_tau - tau_p[p]) % Nu] * 
                     np.exp(1j * 2 * np.pi * epsilon * (n - tilde_tau) / N) 
                     for p in range(P)]) + w_n[n]

# Display results
print("tau_p:")
print(len(tau_p))
print("\nh_tau_p:")
print(len(h_tau_p))
print("\nReceived signal r_n:")
print(len(r_n))


tau_p:
23

h_tau_p:
23

Received signal r_n:
288


In [39]:
# Extract real and imaginary parts and interleave them
y = np.zeros(2 * M)
y[0::2] = np.real(r_n)  # Real parts at even indices
y[1::2] = np.imag(r_n)  # Imaginary parts at odd indices

y = y.reshape(2 * M, 1)

print(y)


[[ 1.45888285e-01]
 [ 1.19357420e+00]
 [-4.26291998e-01]
 [-1.05517285e+00]
 [ 3.27243439e-01]
 [-1.73432469e+00]
 [ 4.09720388e-01]
 [-4.71112902e-01]
 [-3.11853672e+00]
 [-6.92219250e-02]
 [-8.39987613e-01]
 [-7.07798407e-01]
 [ 1.92988206e-02]
 [-1.35024030e+00]
 [ 1.06741711e+00]
 [-3.77955204e-01]
 [-1.20350666e+00]
 [-7.14294369e-01]
 [-5.18586940e-01]
 [-1.14676390e-01]
 [ 1.36420460e+00]
 [ 3.14528708e-01]
 [ 6.43358558e-02]
 [ 5.60031640e-01]
 [-1.14311503e+00]
 [-1.29651315e+00]
 [ 1.36662486e-01]
 [ 2.33591277e+00]
 [ 2.68260511e-01]
 [-1.20881322e+00]
 [-9.82207574e-01]
 [ 6.60323058e-01]
 [-1.89200820e+00]
 [-9.84458362e-01]
 [ 8.22296594e-01]
 [ 1.05965311e-01]
 [-1.25774799e-01]
 [ 1.44281145e-01]
 [-1.59770217e-01]
 [-1.57553055e+00]
 [ 1.56919503e-01]
 [-1.57006318e+00]
 [-1.06462123e+00]
 [-6.30200265e-01]
 [-8.87911143e-01]
 [ 1.66147218e+00]
 [-2.46296325e+00]
 [ 4.07822891e-01]
 [ 1.70842224e+00]
 [ 1.24940065e+00]
 [ 5.62076958e-01]
 [ 1.73193554e+00]
 [-1.2377330